In [34]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API KEY")
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API KEY")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API KEY")
os.environ["LANGSMITH_TRACING"] = "true"

In [3]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini" , model_provider = "openai") 


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_template = ChatPromptTemplate.from_messages([
    ("system", "I am {age} years old. You will act as a scientist expert in Artificail Intelligence"),
    MessagesPlaceholder(variable_name = "messages")
])

In [32]:
from langgraph.graph import START,MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from typing import Sequence
from typing_extensions import TypedDict, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

web_search = TavilySearchResults(max_results=2)
tools = [web_search]
tool_executer = create_react_agent(model,tools)


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    age: str

workflow = StateGraph(state_schema = State)

def call_model(State):
    template = message_template.invoke(State)
    responce = model.invoke(template)
    return {"messages" : responce}

memory = MemorySaver()

workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

app = workflow.compile(checkpointer = memory)

In [33]:
# We can add thread id using a reference also but I dont remeber yet exact syntax of it

config = {"configurable" : {"thread_id" : "mudassar"}}

from langchain_core.messages import HumanMessage
query = "Hello my name is Mudassar Rehman"
age = "25"
input_prompt = [HumanMessage(query)]
output = app.invoke(
    {"messages" : input_prompt, "age": age} , config
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Mudassar Rehman! It's great to meet you. How can I assist you today in the field of artificial intelligence?
